<a href="https://colab.research.google.com/github/DhilipSanjay/Human-Biomechanic-Analysis/blob/main/Human_Biomechanic_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Biomechanic Analysis

## Import necessary libraries


In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import random
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
PD_model = tf.keras.models.load_model("/content/drive/MyDrive/College Related/Project Work 1 - Sem 7/Saved Models/Final Models/PD_Final_Model.h5")
HY_model = tf.keras.models.load_model("/content/drive/MyDrive/College Related/Project Work 1 - Sem 7/Saved Models/Final Models/HY_Final_Model.h5")
FOG_model = tf.keras.models.load_model("/content/drive/MyDrive/College Related/Project Work 1 - Sem 7/Saved Models/Final Models/FOG_Final_Model.h5")

Mounted at /content/drive


## Physionet Dataset

In [ ]:
drive.mount('/content/drive')
dataset_dir = "/content/drive/MyDrive/College Related/Project Work 1 - Sem 7/Dataset/"
dataset_zip = dataset_dir + "gait-in-parkinsons-disease-1.0.0.zip"

!unzip -q "$dataset_zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mv gait-in-parkinsons-disease-1.0.0 Physionet
len(os.listdir('Physionet'))

312

In [ ]:
features = ['Time', 'L1' , 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 
            'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 
            'Total_Force_Left', 'Total_Force_Right']

In [ ]:
!mkdir CSV

# reading given csv file 
# and creating dataframe
for name in os.listdir('Physionet'):
  if 'Co' in name or 'Pt' in name:
    # print(name)
    df = pd.read_csv('Physionet/' + name, header = None, sep='\t')
      
    # adding column headings
    df.columns = features
      
    # store dataframe into csv file
    name = 'CSV/' + name.split('.')[0]+'.csv'
    # print(name)
    df.to_csv(name, index = None)

In [ ]:
demographics = pd.read_csv('Physionet/demographics.txt', delim_whitespace=True)
sub_id = demographics.ID.to_list()
sub_names = []
for name in os.listdir('CSV'):
  sub_name = name.split('_')[0]
  sub_names.append(sub_name)

print("Subjects Count (in demographics) = ", len(sub_id))
print("Files count = ", len(sub_names))

Subjects Count (in demographics) =  166
Files count =  306


In [ ]:
demographics.HoehnYahr
demographics['HoehnYahr'] = demographics['HoehnYahr'].fillna(0)
demographics.HoehnYahr.value_counts()

0.0    73
2.0    55
2.5    28
3.0    10
Name: HoehnYahr, dtype: int64

In [ ]:
demographics['HoehnYahr'].replace({2.5: 1}, inplace=True)
demographics['HoehnYahr'] = demographics['HoehnYahr'].astype(int)
demographics.HoehnYahr.value_counts()

0    73
2    55
1    28
3    10
Name: HoehnYahr, dtype: int64

In [ ]:
!mkdir Final
!mkdir Final/train
!mkdir Final/test

count = len(os.listdir('CSV'))
train_count = int(70/100*count)
test_count = count - train_count

print("Training Count Subjects = ", train_count)
print("Test Count Subjects = ", test_count)

category = 'train'
counter = 1

ypath = 'Final/y_' + category + '.txt'
yfile = open(ypath, "a")
hypath = 'Final/hyscore_' + category + '.txt'
hyfile = open(hypath, "a")

for name in os.listdir('CSV'):
  if counter == train_count + 1:
    yfile.flush()
    yfile.close()
    hyfile.flush()
    hyfile.close()
    
    category = 'test'
    ypath = 'Final/y_' + category + '.txt'
    yfile = open(ypath, "a")
    hypath = 'Final/hyscore_' + category + '.txt'
    hyfile = open(hypath, "a")
  print(counter, name, category)
  
  sub_name = name.split('_')[0]
  sub_class = demographics[demographics['ID'] == sub_name]['Group'].to_string(index=False).strip()
  hy_class = demographics[demographics['ID'] == sub_name]['HoehnYahr'].to_string(index=False).strip()
  sub_data = pd.read_csv('CSV/' + name)
  features = sub_data.columns.to_list()

  full_size = 100
  overlap = 0.5
  overlap_size = int(full_size * overlap / 2)
  entry_size = full_size - overlap_size
  
  for i in range(0, sub_data.shape[0], entry_size):
    if sub_data.shape[0] >= i + entry_size + overlap_size:
      yfile.write(sub_class + "\n")
      hyfile.write(hy_class + "\n")
      
      for fname in features:
        path_name = 'Final/' + category + '/'
        file_name = fname + '_' + category + '.txt'
        with open(path_name + file_name, 'a') as feat_file:
          arr = sub_data.iloc[i:i + entry_size + overlap_size, ][fname].to_numpy()
          # print(i, len(arr), end = ' | ')
          feat_file.write(' '.join(map(str, arr)) + "\n")

  counter = counter + 1
yfile.flush()
yfile.close()
hyfile.flush()
yfile.close()

Training Count Subjects =  214
Test Count Subjects =  92
1 GaCo03_02.csv train
2 GaCo16_02.csv train
3 SiCo24_01.csv train
4 JuPt01_05.csv train
5 JuPt21_05.csv train
6 JuPt23_07.csv train
7 SiCo27_01.csv train
8 SiPt19_01.csv train
9 SiCo26_01.csv train
10 GaPt25_02.csv train
11 GaPt33_02.csv train
12 GaCo12_01.csv train
13 JuPt11_06.csv train
14 GaCo22_01.csv train
15 GaPt16_10.csv train
16 JuPt23_02.csv train
17 SiPt33_01.csv train
18 SiCo17_01.csv train
19 JuCo08_01.csv train
20 JuCo12_01.csv train
21 SiCo05_01.csv train
22 JuPt29_02.csv train
23 JuPt21_07.csv train
24 JuPt15_06.csv train
25 GaPt24_02.csv train
26 GaCo06_01.csv train
27 JuPt06_06.csv train
28 JuPt11_07.csv train
29 SiPt09_01.csv train
30 SiPt37_01.csv train
31 SiCo28_01.csv train
32 JuPt06_07.csv train
33 JuPt15_03.csv train
34 SiCo08_01.csv train
35 JuPt10_04.csv train
36 JuPt09_05.csv train
37 JuPt23_06.csv train
38 SiCo15_01.csv train
39 GaPt15_01.csv train
40 GaPt26_01.csv train
41 SiPt34_01.csv train
42 SiPt27

In [ ]:
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files into a 3D array of [samples, timesteps, features]
def load_category(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded


# load a dataset category, such as train or test
def load_dataset_category(category, prefix=''):
	# load all 19 files as a single array
	filenames = []
	for fname in features:
		filenames.append(category + '/' + fname + '_' + category + '.txt')
  
	# load input data
	X = load_category(filenames, prefix)
	# load class output
	y = load_file(prefix + 'y_'+ category +'.txt')
	hy = load_file(prefix + 'hyscore_'+ category +'.txt')
	return X, y, hy

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy, trainHY = load_dataset_category('train', prefix)
	print(trainX.shape, trainy.shape, trainHY.shape)
	# load all test
	testX, testy, testHY = load_dataset_category('test', prefix)
	print(testX.shape, testy.shape, testHY.shape)
 
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = tf.keras.utils.to_categorical(trainy)
	trainHY = tf.keras.utils.to_categorical(trainHY)
	testy = tf.keras.utils.to_categorical(testy)
	testHY = tf.keras.utils.to_categorical(testHY)  
 
	# print(trainX.shape, trainy.shape, trainHY.shape, testX.shape, testy.shape, testHY.shape)
	return trainX, trainy, trainHY, testX, testy, testHY

In [ ]:
trainX, trainy, trainHY, testX, testy, testHY = load_dataset('Final/')
print("Training Data = ", trainX.shape)
print("Training Class = ", trainy.shape)
print("Training HoehnYahr Class = ", trainHY.shape)
print("Test Data = ", testX.shape)
print("Test Class = ", testy.shape)
print("Test HoehnYahr Class = ", testHY.shape)

(30930, 100, 19) (30930, 1) (30930, 1)
(13096, 100, 19) (13096, 1) (13096, 1)
Training Data =  (30930, 100, 19)
Training Class =  (30930, 2)
Training HoehnYahr Class =  (30930, 4)
Test Data =  (13096, 100, 19)
Test Class =  (13096, 2)
Test HoehnYahr Class =  (13096, 4)


## Daphnet Dataset

In [ ]:
drive.mount('/content/drive')
fog_dataset_dir = "/content/drive/MyDrive/College Related/Project Work 1 - Sem 7/Dataset/"
fog_dataset_zip = fog_dataset_dir + "dataset_fog_release.zip"

!unzip -q "$fog_dataset_zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class_var = 'annontations'
features = ['Time', 'ankle-x', 'ankle-y', 'ankle-z', 
            'thigh-x', 'thigh-y', 'thigh-z',
            'trunk-x', 'trunk-y', 'trunk-z', class_var]
len(os.listdir('dataset_fog_release/dataset'))

17

In [ ]:
!mkdir FOG_CSV

# reading given csv file 
# and creating dataframe
for name in os.listdir('dataset_fog_release/dataset'):
    fog_df = pd.read_csv('dataset_fog_release/dataset/' + name, header = None, sep=' ')
      
    # adding column headings
    fog_df.columns = features
    fog_df = fog_df[fog_df.annontations != 0]  

    # store dataframe into csv file
    name = 'FOG_CSV/' + name.split('.')[0]+'.csv'
    # print(name)
    fog_df.to_csv(name, index = None)

In [ ]:
!mkdir FOG_Final
!mkdir FOG_Final/train
!mkdir FOG_Final/test

count = len(os.listdir('FOG_CSV'))

train_count = int(85/100*count)
test_count = count - train_count

print("Training Count Subjects = ", train_count)
print("Test Count Subjects = ", test_count)

category = 'train'
counter = 1

ypath = 'FOG_Final/y_' + category + '.txt'
yfile = open(ypath, "a")

for name in sorted(os.listdir('FOG_CSV')):
  if counter == train_count + 1:
    yfile.flush()
    yfile.close()
    
    category = 'test'    
    ypath = 'FOG_Final/y_' + category + '.txt'
    yfile = open(ypath, "a")
  print(counter, name, category)
  
  sub_name = name.split('R')[0]
  sub_data = pd.read_csv('FOG_CSV/' + name)
  features = sub_data.columns.to_list()
  
  full_size = 128
  overlap = 0.5
  overlap_size = int(full_size * overlap / 2)
  entry_size = full_size - overlap_size

  for i in range(0, sub_data.shape[0], entry_size):
    if sub_data.shape[0] >= i + entry_size + overlap_size:
      sub_classes = sub_data.iloc[i:i+entry_size+overlap_size, ][class_var].to_list()
      sub_class = max(sub_classes, key = sub_classes.count)
      yfile.write(str(sub_class) + "\n")
 
      for fname in features:
        if fname != 'Time' and fname != class_var:
          path_name = 'FOG_Final/' + category + '/'
          file_name = fname + '_' + category + '.txt'
          with open(path_name + file_name, 'a') as feat_file:
            arr = sub_data.iloc[i:i+entry_size+overlap_size, ][fname].to_numpy()
            # print(i, len(arr), end = ' | ')
            feat_file.write(' '.join(map(str, arr)) + "\n")
  counter = counter + 1
yfile.flush()
yfile.close()

Training Count Subjects =  14
Test Count Subjects =  3
1 S01R01.csv train
2 S01R02.csv train
3 S02R01.csv train
4 S02R02.csv train
5 S03R01.csv train
6 S03R02.csv train
7 S03R03.csv train
8 S04R01.csv train
9 S05R01.csv train
10 S05R02.csv train
11 S06R01.csv train
12 S06R02.csv train
13 S07R01.csv train
14 S07R02.csv train
15 S08R01.csv test
16 S09R01.csv test
17 S10R01.csv test


In [ ]:
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files into a 3D array of [samples, timesteps, features]
def load_category(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded

# load a dataset category, such as train or test
def load_dataset_category(category, prefix=''):
	# load all 10 files as a single array
  filenames = []
  for fname in features:
    if fname != 'Time' and fname != class_var:
      filenames.append(category + '/' + fname + '_' + category + '.txt')
  
  # load input data
  X = load_category(filenames, prefix)
  # load class output
  y = load_file(prefix + 'y_'+ category +'.txt')
  return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
  # load all train
  trainX, trainy = load_dataset_category('train', prefix)
  print(trainX.shape, trainy.shape)
  # load all test
  testX, testy = load_dataset_category('test', prefix)
  print(testX.shape, testy.shape)

  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1

  neg, pos = np.bincount(trainy.reshape(1, trainy.shape[0])[0])
  total = neg + pos
  print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))
  # one hot encode y
  trainy = tf.keras.utils.to_categorical(trainy)
  testy = tf.keras.utils.to_categorical(testy)

  # print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy, neg, pos, total

In [ ]:
fog_trainX, fog_trainy, fog_testX, fog_testy, neg, pos, total = load_dataset('FOG_Final/')

In [ ]:
print("Training Data = ", fog_trainX.shape)
print("Training Class = ", fog_trainy.shape)
print("Test Data = ", fog_testX.shape)
print("Test Class = ", fog_testy.shape)

Training Data =  (8713, 128, 9)
Training Class =  (8713, 2)
Test Data =  (3158, 128, 9)
Test Class =  (3158, 2)


## Final Prediction

In [ ]:
def final_prediction(PD_model, HY_model, FOG_model, pd_data, pd_class, fog_data, fog_class):
  # pd_data - 100 x 19
  # fog_data - 128 x 9
  pd_pred = np.argmax(PD_model.predict(np.expand_dims(pd_data, axis=0))) 
  hy_pred = np.argmax(HY_model.predict(np.expand_dims(pd_data, axis=0)))
  fog_pred = np.argmax(FOG_model.predict(np.expand_dims(fog_data, axis=0)))

  print("ORIGINAL CLASS")
  if np.argmax(pd_class) == 0:
      print("Parkinson's Disease")
  else:
    print("Control")
  
  if np.argmax(fog_class) == 0:
      print("No Freeze")
  else:
    print("Freeze")

  print("-"*20)

  '''
    PD - 0 
    Control - 1
  ------------
    No Freeze - 0
    Freeze - 1
  '''
  
  print("PREDICTION")
  if pd_pred == 0 or fog_pred == 1:
    print("Parkinson's Disease Detected")
    if fog_pred == 1:
      print("Freezing of GAIT")
    else:
      print("No Freezing of GAIT")
  else:
    print("Healthy Control")

  
  '''
  Hoehn Yahr Scale
    0.0	- 0
    2.5	- 1
    2.0	- 2
    3.0	- 3
  '''
  print("Hoehn Yahr Scale Severity: ", end="")
  if hy_pred == 0:
    print("Healthy Control")
  elif hy_pred == 1:
    print("2.5 severity")
  elif hy_pred == 2:
    print("2 severity")
  elif hy_pred == 3:
    print("3 severity")
  
  print("_"*40, "\n")

In [ ]:
for x in range(10):
  i = random.randrange(0, min(len(testX), len(fog_testX)))
  final_prediction(PD_model, HY_model, FOG_model, testX[i], testy[i], fog_testX[i], fog_testy[i])

ORIGINAL CLASS
Parkinson's Disease
No Freeze
--------------------
PREDICTION
Parkinson's Disease Detected
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2 severity
________________________________________ 

ORIGINAL CLASS
Parkinson's Disease
Freeze
--------------------
PREDICTION
Parkinson's Disease Detected
Freezing of GAIT
Hoehn Yahr Scale Severity: 2 severity
________________________________________ 

ORIGINAL CLASS
Control
No Freeze
--------------------
PREDICTION
Healthy Control
Hoehn Yahr Scale Severity: Healthy Control
________________________________________ 

ORIGINAL CLASS
Control
Freeze
--------------------
PREDICTION
Parkinson's Disease Detected
Freezing of GAIT
Hoehn Yahr Scale Severity: Healthy Control
________________________________________ 

ORIGINAL CLASS
Parkinson's Disease
No Freeze
--------------------
PREDICTION
Parkinson's Disease Detected
No Freezing of GAIT
Hoehn Yahr Scale Severity: 2.5 severity
________________________________________ 

ORIGINAL CLASS
Control